In [1]:
import polars as pl

# Expressions
Expressions are functions that map a Series to a Series:
`fn(Series) -> Series`


* Expressions are lazily evaluated
    - Can be optimized by the query optimizer
* Expressions within the same method (e.g. `select`,`with_columns` or `agg`) are evaluated in parallel
* The input Series to an expression is context dependent
    - In the *selection* (filtering)  or *projection* (selecting columns) the input Series is a column or a literal value
    - In the groupby-aggregation context the input Series is the Series for that group


In [2]:
df = pl.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "fruits": ["banana", "banana", "apple", "apple", "banana"],
        "B": [5, 4, 3, 2, 1],
        "cars": ["beetle", "audi", "beetle", "beetle", "beetle"],
        "optional": [28, 300, None, 2, -30],
    }
)
df

A,fruits,B,cars,optional
i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28
2,"""banana""",4,"""audi""",300
3,"""apple""",3,"""beetle""",null
4,"""apple""",2,"""beetle""",2
5,"""banana""",1,"""beetle""",-30


# Selection context

In [32]:
# Within select, we can use the col function to refer to columns.
# If we are not applying any function to the column, we can also use the column name as a string.

(
    df
    .select(
        [
            pl.col("A"),
            "B",      # the col part is inferred
            pl.lit("B"),  # the pl.lit functions tell polars we mean the literal "B"
        ]
    )
)

A,B,literal
i64,i64,str
1,5,"""B"""
2,4,"""B"""
3,3,"""B"""
4,2,"""B"""
5,1,"""B"""


In [ ]:
# We can use a list within select (example above) or a comma-separated list of expressions (this example).

(
    df
    .select(
        pl.col("A"),
        "B",      
        pl.lit("B"),  
    )
)

In [4]:
# We can select columns with a regex if the regex starts with '^' and ends with '$'

(
    df
    .select(    
        pl.col("^A|B$").sum()
    )
)

A,B
i64,i64
15,15


In [5]:
# We can select multiple columns by name

(
    df
    .select(
        pl.col(["A", "B"]).sum()
    )
)

A,B
i64,i64
15,15


In [6]:
# We select everything in normal order
# Then we select everything in reversed order

(
    df
    .select(
        pl.all(),
        pl.all().reverse().suffix("_reverse")
    )
)

A,fruits,B,cars,optional,A_reverse,fruits_reverse,B_reverse,cars_reverse,optional_reverse
i64,str,i64,str,i64,i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28,5,"""banana""",1,"""beetle""",-30
2,"""banana""",4,"""audi""",300,4,"""apple""",2,"""beetle""",2
3,"""apple""",3,"""beetle""",null,3,"""apple""",3,"""beetle""",null
4,"""apple""",2,"""beetle""",2,2,"""banana""",4,"""audi""",300
5,"""banana""",1,"""beetle""",-30,1,"""banana""",5,"""beetle""",28


In [7]:
# All expressions run in parallel
# Single valued `Series` are broadcasted to the shape of the `DataFrame`

(
    df
    .select(
        pl.all(),
        pl.all().sum().suffix("_sum") # This is a single valued Series broadcasted to the shape of the DataFrame
    )
)

A,fruits,B,cars,optional,A_sum,fruits_sum,B_sum,cars_sum,optional_sum
i64,str,i64,str,i64,i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28,15,null,15,null,300
2,"""banana""",4,"""audi""",300,15,null,15,null,300
3,"""apple""",3,"""beetle""",null,15,null,15,null,300
4,"""apple""",2,"""beetle""",2,15,null,15,null,300
5,"""banana""",1,"""beetle""",-30,15,null,15,null,300


In [33]:
# There are `str` and `dt` namespaces for specialized functions
# For example, we can select all rows where the `fruits` column starts with 'b'

predicate = pl.col("fruits").str.contains("^b.*")

(
    df
    .filter(
        predicate
    )
)

A,fruits,B,cars,optional
i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28
2,"""banana""",4,"""audi""",300
5,"""banana""",1,"""beetle""",-30


In [30]:
# Filters can also be applied within an expression
(
    df
    .select(
        # Sum the values of A where the fruits column starts with 'b'
        pl.col("A").filter(pl.col("fruits").str.contains("^b.*")).sum(),
    )
)

A
i64
8


In [11]:
# We can do arithmetic on columns

(
    df
    .select(
        (
            (pl.col("A") / 124.0 * pl.col("B")) / pl.sum("B")
        ).alias("computed")
    )
)

computed
f64
0.002688
0.004301
0.004839
0.004301
0.002688


In [12]:
# We can combine columns by a predicate
# For example when the `fruits` column is 'banana' we set the value equal to the
# value in `B` column for that row, otherwise we set the value to be -1

(
    df
    .select(
        "fruits",
        "B",
        pl.when(
            pl.col("fruits") == "banana"
        )
        .then(
            pl.col("B"))
        .otherwise(-1)
        .alias("b")
    )
)

fruits,B,b
str,i64,i64
"""banana""",5,5
"""banana""",4,4
"""apple""",3,-1
"""apple""",2,-1
"""banana""",1,1


In [26]:
# We can combine columns by a fold operation on column level.
# For example we do a horizontal sum where we:
# - start with 0
# - add the value in the `A` column
# - add the value in the `B` column
# - add the value in the `B` column squared

(
    df
    .select(
        [
            "A",
            "B",
            pl.fold(
                0, 
                lambda a, b: a + b, 
                [
                    pl.col("A"), "B", pl.col("B")**2,
                ]
            )
            .alias("fold")
        ]
    )
)

A,B,fold
i64,i64,f64
1,5,31.0
2,4,22.0
3,3,15.0
4,2,10.0
5,1,7.0


# Aggregation context
In the aggregation context expressions are applied over groups instead of columns.

We pass the expressions as a list to .agg. If we aggregate on the same column multiple times we must use alias on at least one of them to avoid a column name collision


In [ ]:

(
    df
    .groupby("fruits")
    .agg(
        [
            pl.col("B").count().alias("B_count"),
            pl.col("B").sum().alias("B_sum")
        ]
    )
)

In [27]:
# We can aggregate many expressions at once

(
    df
    .groupby("fruits")
    .agg(
        [
            pl.col("B").sum().alias("B_sum"),# Sum of B
            pl.first("fruits").alias("fruits_first"),# First value of fruits
            pl.count("A").alias("count"),# Count of A
            pl.col("cars").reverse() # Reverse the cars column - not an aggregation
            # so the output is a pl.List
        ]
    )
)

fruits,B_sum,B_sum2,fruits_first,count,cars
str,i64,i64,str,u32,list[str]
"""banana""",10,10,"""banana""",3,"[""beetle"", ""audi"", ""beetle""]"
"""apple""",5,5,"""apple""",2,"[""beetle"", ""beetle""]"


In [16]:
# We can explode the list column "cars" to a new row for each element in the list

(
    df
    .sort("cars")
    .groupby("fruits")
    .agg(
        [
            pl.col("B").sum().alias("B_sum"),
            pl.sum("B").alias("B_sum2"),  # syntactic sugar for the first
            pl.first("fruits").alias("fruits_first"),
            pl.count("A").alias("count"),
            pl.col("cars").reverse()
        ]
    )
    .explode("cars")
)


fruits,B_sum,B_sum2,fruits_first,count,cars
str,i64,i64,str,u32,str
"""banana""",10,10,"""banana""",3,"""beetle"""
"""banana""",10,10,"""banana""",3,"""beetle"""
"""banana""",10,10,"""banana""",3,"""audi"""
"""apple""",5,5,"""apple""",2,"""beetle"""
"""apple""",5,5,"""apple""",2,"""beetle"""


In [28]:
# We can also get a list of the row indices for each group with `agg_groups()`

(
    df
    .groupby("fruits")
    .agg(
         pl.col("B").agg_groups().alias("group_row_indices"),
     )
)

fruits,group_row_indices
str,list[u32]
"""apple""","[2, 3]"
"""banana""","[0, 1, 4]"


In [29]:
# We can also do filter predicates in groupby
# In this example we do not include values of B that are smaller than 1
# in the sum
(
    df
    .groupby("fruits")
    .agg(
        [
            pl.col("B").filter(pl.col("B") > 1).sum(),
        ]
    )
)

fruits,B
str,i64
"""apple""",5
"""banana""",9


# Window functions

Window functions allow us to call expressions on groups of rows. We can then add a column with values based on groups of rows.

Window functions allow us to do aggregation in selection context (i.e. keeping all the rows of the original `DataFrame`).

The basic pattern for window functions is:
```python
pl.col("foo").aggregation_expression(..).over("column_used_to_group")
```


In [31]:
# Here we add a new column with the sum of B grouped by fruits

(
    df
    .select(
        "fruits",
        "cars",
        "B",
        pl.col("B").sum().over("fruits").alias("B_sum_by_fruits"),
)
)

fruits,cars,B,B_sum_by_fruits
str,str,i64,i64
"""banana""","""beetle""",5,10
"""banana""","""audi""",4,10
"""apple""","""beetle""",3,5
"""apple""","""beetle""",2,5
"""banana""","""beetle""",1,10


In [23]:
# We can also use window functions to do groupby over multiple columns

(
    df
    .select(
        "fruits",
        "cars",
        "B",
        pl.col("B").sum().over("fruits").alias("B_sum_by_fruits"),
        pl.col("B").sum().over("cars").alias("B_sum_by_cars"),
    )
)

fruits,cars,B,B_sum_by_fruits,B_sum_by_cars
str,str,i64,i64,i64
"""banana""","""beetle""",5,10,11
"""banana""","""audi""",4,10,4
"""apple""","""beetle""",3,5,11
"""apple""","""beetle""",2,5,11
"""banana""","""beetle""",1,10,11


In [25]:
# Here we use a window function to lag column B within "fruits"

(
    df
    .select(
        "fruits",
        "B",
        pl.col("B").shift().over("fruits").alias("lag_B_by_fruits")
    )
)

fruits,B,lag_B_by_fruits
str,i64,i64
"""banana""",5,null
"""banana""",4,5
"""apple""",3,null
"""apple""",2,3
"""banana""",1,4
